<font color="#CA0032"><h1 align="left">**Cajamar Water Footprint**</h1></font>

<font color="#6E6E6E"><h1 align="left">**Predicción de series temporales**</h1></font>

<h2 align="left">Tur-IA</h2>

## Importar librerías

In [195]:
!pip install --upgrade mxnet==1.6.0
!pip install gluonts

In [196]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score as R2_score

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator


from gluonts.model.deepar import DeepAREstimator
from gluonts.mx import Trainer

import time

%matplotlib inline
# fijo la semilla aleatoria por reproducibilidad
np.random.seed(123)

## Carga de datos

In [197]:
data = pd.read_csv('df_colab_daily_nan.csv', sep='|', index_col=0)
data

,2019-02-01,2019-02-02,2019-02-03,2019-02-04,2019-02-05,2019-02-06,2019-02-07,2019-02-08,2019-02-09,2019-02-10,...,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31
ID,,,,,,,,,,,,,,,,,,,,,
0,243.00,236.00,335.00,252.0,220.0,276.00,277.00,193.00,262.00,315.00,...,460.0,331.0,368.00,390.00,304.00,213.0,232.0,403.00,425.0,255.00
1,8.00,47.00,6.00,12.0,44.0,0.00,6.99,0.00,92.06,62.00,...,0.0,3.0,5.61,1.32,2.97,4.0,1.0,1.98,10.0,32.95
2,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,...,33.0,20.0,37.00,30.00,32.00,39.0,40.0,45.00,27.0,59.00
3,492.00,381.00,313.00,362.0,380.0,453.00,390.00,419.00,459.00,442.00,...,312.0,321.0,439.00,639.00,350.00,289.0,380.0,404.00,339.0,391.00
4,247.56,235.68,254.35,412.0,269.0,224.52,218.25,429.25,306.00,194.97,...,287.0,361.0,203.00,272.04,281.98,263.6,303.4,421.00,420.0,253.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2746,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2747,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [198]:
data.shape

(2747, 365)

In [199]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2747 entries, 0 to 2756
Columns: 365 entries, 2019-02-01 to 2020-01-31
dtypes: float64(365)
memory usage: 7.7 MB


In [200]:
data.isnull().sum(axis=1)

ID
0         1
1         9
2         5
3         1
4         3
       ... 
2746    362
2747    364
2748    361
2749    363
2756    364
Length: 2747, dtype: int64

## Preprocesado

In [201]:
data.reset_index(inplace=True)
data

,ID,2019-02-01,2019-02-02,2019-02-03,2019-02-04,2019-02-05,2019-02-06,2019-02-07,2019-02-08,2019-02-09,...,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31
0,0,243.00,236.00,335.00,252.0,220.0,276.00,277.00,193.00,262.00,...,460.0,331.0,368.00,390.00,304.00,213.0,232.0,403.00,425.0,255.00
1,1,8.00,47.00,6.00,12.0,44.0,0.00,6.99,0.00,92.06,...,0.0,3.0,5.61,1.32,2.97,4.0,1.0,1.98,10.0,32.95
2,2,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,...,33.0,20.0,37.00,30.00,32.00,39.0,40.0,45.00,27.0,59.00
3,3,492.00,381.00,313.00,362.0,380.0,453.00,390.00,419.00,459.00,...,312.0,321.0,439.00,639.00,350.00,289.0,380.0,404.00,339.0,391.00
4,4,247.56,235.68,254.35,412.0,269.0,224.52,218.25,429.25,306.00,...,287.0,361.0,203.00,272.04,281.98,263.6,303.4,421.00,420.0,253.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2742,2746,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2743,2747,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2744,2748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2745,2749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [202]:
num_cont = len(data['ID'].unique())
num_muestras = len(data.columns) - 1

print(num_cont, num_muestras)

2747 365


In [203]:
df_input = data

In [204]:
ts_code=df_input["ID"].astype('category').cat.codes.values
ts_code

array([   0,    1,    2, ..., 2744, 2745, 2746], dtype=int16)

In [205]:
N_val = 14
freq="1D"
window_size=90 
prediction_lentgh=N_val

In [206]:
df_aux = pd.DataFrame(data=None, index=df_input.index, columns=['2020-02-01',
                                                                '2020-02-02',
                                                                '2020-02-03',
                                                                '2020-02-04',
                                                                '2020-02-05',
                                                                '2020-02-06',
                                                                '2020-02-07',
                                                                '2020-02-08',
                                                                '2020-02-09',
                                                                '2020-02-10',
                                                                '2020-02-11',
                                                                '2020-02-12',
                                                                '2020-02-13',
                                                                '2020-02-14'])
df_aux.fillna(0, inplace=True)
df_aux

,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2742,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2743,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2744,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2745,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [207]:
df_aux = df_input.join(df_aux)
df_aux

,ID,2019-02-01,2019-02-02,2019-02-03,2019-02-04,2019-02-05,2019-02-06,2019-02-07,2019-02-08,2019-02-09,...,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14
0,0,243.00,236.00,335.00,252.0,220.0,276.00,277.00,193.00,262.00,...,0,0,0,0,0,0,0,0,0,0
1,1,8.00,47.00,6.00,12.0,44.0,0.00,6.99,0.00,92.06,...,0,0,0,0,0,0,0,0,0,0
2,2,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,...,0,0,0,0,0,0,0,0,0,0
3,3,492.00,381.00,313.00,362.0,380.0,453.00,390.00,419.00,459.00,...,0,0,0,0,0,0,0,0,0,0
4,4,247.56,235.68,254.35,412.0,269.0,224.52,218.25,429.25,306.00,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2742,2746,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2743,2747,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2744,2748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2745,2749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [208]:
df_train = df_input.iloc[:,1:-N_val].values
df_val = df_input.iloc[:,1:].values
df_test =df_aux.iloc[:,1:].values

print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(2747, 351)
(2747, 365)
(2747, 379)


## Modelo

In [209]:
from gluonts.transform import MeanValueImputation, RollingMeanValueImputation

In [210]:
estimator = DeepAREstimator(freq=freq, 
                            context_length=window_size,
                            prediction_length=prediction_lentgh,
                            use_feat_static_cat=True,
                            cardinality=[1],
                            num_layers=2,
                            num_cells=72,
                            cell_type='lstm',
                            impute_missing_values=True,
                            num_imputation_samples=10,
                            imputation_method=RollingMeanValueImputation(),
                            trainer=Trainer(epochs=2, learning_rate=0.005))

In [211]:
start_train = df_input.columns[1:].min()

In [212]:
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName

train_ds = ListDataset([
    {
        FieldName.TARGET: target,
        FieldName.START: start_train,
        FieldName.FEAT_STATIC_CAT: fsc
    }
    for (target, fsc) in zip(df_train, ts_code.reshape(-1,1))
], freq=freq)

In [213]:
val_ds = ListDataset([
    {
        FieldName.TARGET: target,
        FieldName.START: start_train,
        FieldName.FEAT_STATIC_CAT: fsc
    }
    for (target, fsc) in zip(df_val, ts_code.reshape(-1,1))
], freq=freq)

In [214]:
test_ds = ListDataset([
    {
        FieldName.TARGET: target,
        FieldName.START: start_train,
        FieldName.FEAT_STATIC_CAT: fsc
    }
    for (target, fsc) in zip(df_test, ts_code.reshape(-1,1))
], freq=freq)

In [215]:
next(iter(train_ds))

{'feat_static_cat': array([0], dtype=int32),
 'source': SourceContext(source='list_data', row=0),
 'start': Timestamp('2019-02-01 00:00:00', freq='D'),
 'target': array([243., 236., 335., 252., 220., 276., 277., 193., 262., 315., 224.,
        337., 219., 264., 384., 279., 343., 320., 410., 259., 313., 381.,
        222., 355., 383., 248., 314., 180., 208., 230., 354., 321., 319.,
        304., 243., 135., 266., 423., 545., 220., 209., 476., 324., 286.,
        225., 215., 409., 284., 380., 189., 293., 384., 228., 169., 287.,
        323., 459., 296., 399., 380., 309., 398., 516., 231., 438., 387.,
        309., 329., 333., 324., 362., 234., 212., 322., 282.,  97., 234.,
          0.,   0.,   0.,   0.,   0.,   0., 404., 364., 254., 201., 286.,
        270., 143., 276., 229., 302., 351., 321., 268., 177., 299., 339.,
        288., 463., 277., 332., 320., 410., 333., 224., 344., 382., 367.,
        376., 130., 134., 127., 336., 409., 208., 319., 143., 268., 243.,
        300., 331., 260.

## Entrenamiendo para predicciones Validación

In [216]:
predictor = estimator.train(training_data=train_ds)

/usr/local/lib/python3.7/dist-packages/gluonts/transform/feature.py:343: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  self._freq_base = start.freq.base
/usr/local/lib/python3.7/dist-packages/gluonts/transform/split.py:36: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  return _shift_timestamp_helper(ts, ts.freq, offset)
/usr/local/lib/python3.7/dist-packages/gluonts/transform/feature.py:386: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  if self._full_range_date_features is not None
/usr/local/lib/python3.7/dist-packages/gluonts/transform/feature.py:340: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  self._freq_base is None or self._freq_base == start.freq.base
  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/gluonts/transform/feature.py:340: FutureWarning: Timestamp.freq is deprecated and will be remove

In [217]:
from gluonts.evaluation.backtest import make_evaluation_predictions

forecast_it, ts_it = make_evaluation_predictions(
    dataset=val_ds,  
    predictor=predictor,  
    num_samples=100, 
)

In [218]:
from tqdm.autonotebook import tqdm

print("Obtaining time series conditioning values ...")
tss = list(tqdm(ts_it, total=len(df_test)))
print("Obtaining time series predictions ...")
forecasts = list(tqdm(forecast_it, total=len(df_test)))

Obtaining time series conditioning values ...


  0%|          | 0/2747 [00:00<?, ?it/s]

Obtaining time series predictions ...


  0%|          | 0/2747 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/gluonts/transform/feature.py:340: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  self._freq_base is None or self._freq_base == start.freq.base
/usr/local/lib/python3.7/dist-packages/gluonts/transform/split.py:36: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  return _shift_timestamp_helper(ts, ts.freq, offset)
/usr/local/lib/python3.7/dist-packages/gluonts/transform/feature.py:386: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  if self._full_range_date_features is not None


In [219]:
# first entry of the time series list
ts_entry = tss[0]

In [220]:
# first 5 values of the time series (convert from pandas to numpy)
np.array(ts_entry[:5]).reshape(-1,)

array([243., 236., 335., 252., 220.], dtype=float32)

In [221]:
# first entry of the forecast list
forecast_entry = forecasts[0]

In [222]:
print(f"Number of sample paths: {forecast_entry.num_samples}")
print(f"Dimension of samples: {forecast_entry.samples.shape}")
print(f"Start date of the forecast window: {forecast_entry.start_date}")
print(f"Frequency of the time series: {forecast_entry.freq}")

Number of sample paths: 100
Dimension of samples: (100, 14)
Start date of the forecast window: 2020-01-18 00:00:00
Frequency of the time series: 1D


In [223]:
def plot_prob_forecasts(ts_entry, forecast_entry):
    plot_length = window_size + prediction_lentgh
    prediction_intervals = (80, 95)
    legend = ["observations", "median prediction"] + [f"{k}% prediction interval" for k in prediction_intervals][::-1]

    fig, ax = plt.subplots(1, 1, figsize=(30, 10))
    ts_entry[-plot_length:].plot(ax=ax)  
    forecast_entry.plot(prediction_intervals=prediction_intervals, color='g')
    plt.grid(which="both")
    plt.legend(legend, loc="upper left")
    plt.show()

In [224]:
import random
random_range = random.choices(range(len(df_train)),k=50)

In [225]:
for i in tqdm(range(50)):
    ts_entry = tss[i]
    forecast_entry = forecasts[i]
    plot_prob_forecasts(ts_entry, forecast_entry)

Output hidden; open in https://colab.research.google.com to view.

In [226]:
from gluonts.evaluation import Evaluator
evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(df_test))

Running evaluation: 100%|██████████| 2747/2747 [00:00<00:00, 6363.49it/s]
/usr/local/lib/python3.7/dist-packages/gluonts/evaluation/_base.py:306: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  date_before_forecast = forecast.index[0] - forecast.index[0].freq
/usr/local/lib/python3.7/dist-packages/gluonts/evaluation/_base.py:306: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  date_before_forecast = forecast.index[0] - forecast.index[0].freq
/usr/local/lib/python3.7/dist-packages/gluonts/evaluation/metrics.py:102: RuntimeWarning: divide by zero encountered in double_scalars
  return np.mean(np.abs(target - forecast)) / seasonal_error
/usr/local/lib/python3.7/dist-packages/gluonts/evaluation/metrics.py:150: RuntimeWarning: divide by zero encountered in double_scalars
  return numerator / seasonal_error
/usr/local/lib/python3.7/dist-packages/gluonts/evaluation/metrics.py:102: RuntimeWarning: divide by zero encoun

In [227]:
item_metrics

,item_id,MSE,abs_error,abs_target_sum,abs_target_mean,seasonal_error,MASE,MAPE,sMAPE,MSIS,QuantileLoss[0.1],Coverage[0.1],QuantileLoss[0.5],Coverage[0.5],QuantileLoss[0.9],Coverage[0.9]
0,NaN,15676.390625,1493.330078,4673.000000,333.785714,96.537356,1.104924,0.288100,0.355258,7.801847,816.532318,0.0,1493.330215,0.214286,423.970135,0.928571
1,NaN,101.891451,72.354225,79.830002,6.140769,8.939791,0.622577,0.798724,1.338731,12.084531,26.135724,0.0,72.354225,0.230769,72.224089,0.769231
2,NaN,244.501465,183.528168,506.000000,36.142857,6.589443,1.989418,0.334287,0.422637,15.309418,104.789428,0.0,183.528162,0.071429,73.346443,0.928571
3,NaN,6042.262277,679.644043,5277.000000,376.928571,93.504484,0.519184,0.118555,0.122822,10.679586,746.676166,0.0,679.644073,0.571429,743.097070,1.000000
4,NaN,10607.073661,1331.726318,4482.020020,320.144287,75.902849,1.253224,0.276732,0.332497,10.125850,751.449127,0.0,1331.726288,0.071429,396.480618,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2742,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2745,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Cáculo RMSE para validación

In [228]:
originales = data.iloc[:,-N_val:]
originales

,2020-01-18,2020-01-19,2020-01-20,2020-01-21,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31
0,421.0,273.0,306.00,292.0,460.0,331.0,368.00,390.00,304.00,213.0,232.0,403.00,425.0,255.00
1,0.0,NaN,14.00,3.0,0.0,3.0,5.61,1.32,2.97,4.0,1.0,1.98,10.0,32.95
2,28.0,33.0,48.00,35.0,33.0,20.0,37.00,30.00,32.00,39.0,40.0,45.00,27.0,59.00
3,485.0,394.0,237.00,297.0,312.0,321.0,439.00,639.00,350.00,289.0,380.0,404.00,339.0,391.00
4,365.0,387.0,370.02,293.0,287.0,361.0,203.00,272.04,281.98,263.6,303.4,421.00,420.0,253.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2742,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2745,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [229]:
originales_1_7 = originales.iloc[:,:7]
originales_1_7 = originales_1_7.values
originales_1_7.shape

(2747, 7)

In [230]:
originales_semana1 = originales_1_7.sum(axis=1)
originales_semana1 = np.array(originales_semana1).reshape(-1,1)
originales_semana1.shape

(2747, 1)

In [231]:
originales_semana2 = originales.iloc[:,7:].sum(axis=1)
originales_semana2 = np.array(originales_semana2).reshape(-1,1)
originales_semana2.shape

(2747, 1)

In [232]:
array_originales = np.concatenate((originales_1_7,originales_semana1,originales_semana2), axis=1)
array_originales

array([[ 421.  ,  273.  ,  306.  , ...,  368.  , 2451.  , 2222.  ],
       [   0.  ,     nan,   14.  , ...,    5.61,     nan,   54.22],
       [  28.  ,   33.  ,   48.  , ...,   37.  ,  234.  ,  272.  ],
       ...,
       [    nan,     nan,     nan, ...,     nan,     nan,    0.  ],
       [    nan,     nan,     nan, ...,     nan,     nan,    0.  ],
       [    nan,     nan,     nan, ...,     nan,     nan,    0.  ]])

In [233]:
array_originales = np.nan_to_num(array_originales,nan=0.0)
array_originales

array([[ 421.  ,  273.  ,  306.  , ...,  368.  , 2451.  , 2222.  ],
       [   0.  ,    0.  ,   14.  , ...,    5.61,    0.  ,   54.22],
       [  28.  ,   33.  ,   48.  , ...,   37.  ,  234.  ,  272.  ],
       ...,
       [   0.  ,    0.  ,    0.  , ...,    0.  ,    0.  ,    0.  ],
       [   0.  ,    0.  ,    0.  , ...,    0.  ,    0.  ,    0.  ],
       [   0.  ,    0.  ,    0.  , ...,    0.  ,    0.  ,    0.  ]])

In [234]:
array_originales.shape

(2747, 9)

In [235]:
array_predicciones = np.empty([num_cont,N_val])
array_predicciones.shape

(2747, 14)

In [236]:
for i in tqdm(range(num_cont)):
    forecast_entry = forecasts[i]
    array_predicciones[i] = forecast_entry.quantile(0.5)

  0%|          | 0/2747 [00:00<?, ?it/s]

In [237]:
array_predicciones

array([[304.78408813, 222.82279968, 263.42300415, ..., 206.60777283,
        177.60017395, 205.69319153],
       [  1.23332739,   1.9646132 ,   3.32594204, ...,   0.40887856,
          0.80398291,   1.04450881],
       [ 25.35119057,  28.06176567,  31.25800705, ...,  18.13321304,
         16.66000175,  16.85003662],
       ...,
       [-51.50085831, -50.64999008, -47.73821259, ..., -49.7697258 ,
        -49.97432709, -49.01314926],
       [-51.72757721, -53.6912117 , -48.41495132, ..., -50.76372147,
        -49.7568512 , -50.79929733],
       [-52.48104858, -54.04344559, -51.5717392 , ..., -52.6150322 ,
        -50.32485199, -53.66513824]])

In [238]:
df_solucion = pd.DataFrame(array_predicciones).fillna(0)
df_solucion

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,304.784088,222.822800,263.423004,326.850891,230.538666,223.137131,214.376678,237.061539,233.039078,226.982635,259.580811,206.607773,177.600174,205.693192
1,1.233327,1.964613,3.325942,2.246829,2.813140,6.256519,3.227249,-0.275486,-0.573252,0.998980,0.571131,0.408879,0.803983,1.044509
2,25.351191,28.061766,31.258007,32.912868,26.841410,26.464685,23.944643,17.250719,23.877031,21.359037,26.436600,18.133213,16.660002,16.850037
3,420.101868,396.428925,306.141846,339.973053,336.627045,343.954987,381.400330,403.646179,392.025513,301.286102,357.279907,336.483673,325.645447,392.764008
4,297.563354,255.508926,224.808029,242.143005,232.275803,236.876968,267.621918,252.907562,245.145630,230.081802,176.363846,228.124512,168.287766,221.828430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2742,-50.687954,-49.140274,-45.049797,-46.659870,-46.025612,-47.736427,-49.681854,-50.669224,-52.533710,-47.021633,-47.559097,-49.839493,-50.061852,-50.077915
2743,-50.869640,-51.339622,-48.043400,-49.963219,-48.302162,-49.459587,-48.604961,-49.432957,-50.234413,-44.797745,-46.577503,-47.688702,-47.219753,-48.924252
2744,-51.500858,-50.649990,-47.738213,-49.277130,-48.306519,-52.525612,-50.686131,-50.647560,-50.391571,-48.977432,-48.545135,-49.769726,-49.974327,-49.013149
2745,-51.727577,-53.691212,-48.414951,-47.946331,-48.366158,-49.285339,-47.555355,-52.568630,-52.036034,-47.535252,-49.200924,-50.763721,-49.756851,-50.799297


In [239]:
df_solucion.clip(lower=0, inplace=True)
df_solucion

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,304.784088,222.822800,263.423004,326.850891,230.538666,223.137131,214.376678,237.061539,233.039078,226.982635,259.580811,206.607773,177.600174,205.693192
1,1.233327,1.964613,3.325942,2.246829,2.813140,6.256519,3.227249,0.000000,0.000000,0.998980,0.571131,0.408879,0.803983,1.044509
2,25.351191,28.061766,31.258007,32.912868,26.841410,26.464685,23.944643,17.250719,23.877031,21.359037,26.436600,18.133213,16.660002,16.850037
3,420.101868,396.428925,306.141846,339.973053,336.627045,343.954987,381.400330,403.646179,392.025513,301.286102,357.279907,336.483673,325.645447,392.764008
4,297.563354,255.508926,224.808029,242.143005,232.275803,236.876968,267.621918,252.907562,245.145630,230.081802,176.363846,228.124512,168.287766,221.828430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2742,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2743,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2744,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2745,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [240]:
predicciones_1_7 = df_solucion.iloc[:,:7]
predicciones_1_7 = predicciones_1_7.values
predicciones_1_7.shape

(2747, 7)

In [241]:
predicciones_semana1 = df_solucion.iloc[:,:7].sum(axis=1)
predicciones_semana1 = np.array(predicciones_semana1).reshape(-1,1)
predicciones_semana1.shape

(2747, 1)

In [242]:
predicciones_semana2 = df_solucion.iloc[:,7:].sum(axis=1)
predicciones_semana2 = np.array(predicciones_semana2).reshape(-1,1)
predicciones_semana2.shape

(2747, 1)

In [243]:
array_predicciones = np.concatenate((predicciones_1_7,predicciones_semana1,predicciones_semana2), axis=1)
array_predicciones

array([[3.04784088e+02, 2.22822800e+02, 2.63423004e+02, ...,
        2.14376678e+02, 1.78593326e+03, 1.54656520e+03],
       [1.23332739e+00, 1.96461320e+00, 3.32594204e+00, ...,
        3.22724891e+00, 2.10676200e+01, 3.82748121e+00],
       [2.53511906e+01, 2.80617657e+01, 3.12580070e+01, ...,
        2.39446430e+01, 1.94834570e+02, 1.40566639e+02],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [247]:
from sklearn.metrics import mean_squared_error

def calcular_rmse(predicciones, originales):
  list_rmse = []

  for i in range(originales.shape[0]):
    rmse_cal = mean_squared_error(originales[i], predicciones[i], squared=False)
    list_rmse.append(rmse_cal)

  return sum(list_rmse)

In [248]:
calcular_rmse(array_predicciones,array_originales)

1331845.102621121

## Entrenamiento para obtener predicciones Test

In [249]:
predictor = estimator.train(training_data=val_ds)

/usr/local/lib/python3.7/dist-packages/gluonts/transform/feature.py:343: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  self._freq_base = start.freq.base
/usr/local/lib/python3.7/dist-packages/gluonts/transform/split.py:36: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  return _shift_timestamp_helper(ts, ts.freq, offset)
/usr/local/lib/python3.7/dist-packages/gluonts/transform/feature.py:386: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  if self._full_range_date_features is not None
/usr/local/lib/python3.7/dist-packages/gluonts/transform/feature.py:340: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  self._freq_base is None or self._freq_base == start.freq.base
  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/gluonts/transform/feature.py:340: FutureWarning: Timestamp.freq is deprecated and will be remove

In [250]:
from gluonts.evaluation.backtest import make_evaluation_predictions

forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_ds,  
    predictor=predictor,  
    num_samples=100, 
)

In [251]:
from tqdm.autonotebook import tqdm

print("Obtaining time series conditioning values ...")
tss = list(tqdm(ts_it, total=len(df_test)))
print("Obtaining time series predictions ...")
forecasts = list(tqdm(forecast_it, total=len(df_test)))

Obtaining time series conditioning values ...


  0%|          | 0/2747 [00:00<?, ?it/s]

Obtaining time series predictions ...


  0%|          | 0/2747 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/gluonts/transform/feature.py:340: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  self._freq_base is None or self._freq_base == start.freq.base
/usr/local/lib/python3.7/dist-packages/gluonts/transform/split.py:36: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  return _shift_timestamp_helper(ts, ts.freq, offset)
/usr/local/lib/python3.7/dist-packages/gluonts/transform/feature.py:386: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  if self._full_range_date_features is not None


In [252]:
# first entry of the time series list
ts_entry = tss[0]

In [253]:
# first 5 values of the time series (convert from pandas to numpy)
np.array(ts_entry[:5]).reshape(-1,)

array([243., 236., 335., 252., 220.], dtype=float32)

In [254]:
# first entry of the forecast list
forecast_entry = forecasts[0]

In [255]:
print(f"Number of sample paths: {forecast_entry.num_samples}")
print(f"Dimension of samples: {forecast_entry.samples.shape}")
print(f"Start date of the forecast window: {forecast_entry.start_date}")
print(f"Frequency of the time series: {forecast_entry.freq}")

Number of sample paths: 100
Dimension of samples: (100, 14)
Start date of the forecast window: 2020-02-01 00:00:00
Frequency of the time series: 1D


In [256]:
for i in tqdm(range(50)):
    ts_entry = tss[i]
    forecast_entry = forecasts[i]
    plot_prob_forecasts(ts_entry, forecast_entry)

Output hidden; open in https://colab.research.google.com to view.

## Postprocesado para guardar los datos

In [257]:
print(num_cont)
print(N_val)

2747
14


In [258]:
array_predicciones = np.empty([num_cont,N_val])
array_predicciones.shape

(2747, 14)

In [259]:
for i in tqdm(range(num_cont)):
    forecast_entry = forecasts[i]
    array_predicciones[i] = forecast_entry.quantile(0.5)

  0%|          | 0/2747 [00:00<?, ?it/s]

In [260]:
array_predicciones

array([[337.45501709, 278.82409668, 275.38928223, ..., 339.40951538,
        317.94927979, 281.16897583],
       [  7.74267387,   4.52196598,   6.74253893, ...,   2.1664691 ,
          5.36327505,   6.58795309],
       [ 33.24901199,  34.91620636,  35.37581635, ...,  25.47824287,
         24.67741203,  27.4685154 ],
       ...,
       [  9.27019691,  15.05194473,  43.2884903 , ...,  39.00149918,
         42.57069397,   4.5509882 ],
       [-30.44287491, -14.45410061,  -5.56974411, ...,   1.14718926,
        -20.52612114, -28.86708641],
       [-51.72711563, -20.02453613,   4.57386017, ...,  12.85540485,
          6.99222708,   3.32102633]])

In [261]:
array_predicciones.shape

(2747, 14)

In [262]:
df_solucion = pd.DataFrame(array_predicciones)
df_solucion

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,337.455017,278.824097,275.389282,332.793732,334.838531,296.068909,285.253143,281.926971,276.862732,268.698914,334.197815,339.409515,317.949280,281.168976
1,7.742674,4.521966,6.742539,4.463351,0.957877,5.990738,7.986382,3.660743,1.675190,3.162817,3.271239,2.166469,5.363275,6.587953
2,33.249012,34.916206,35.375816,30.073835,29.788906,26.057459,33.799553,26.341917,28.113489,30.577930,28.227587,25.478243,24.677412,27.468515
3,445.453156,344.273193,336.236267,355.775665,403.568481,366.044708,405.181213,447.589172,375.764191,356.018524,357.474701,429.301514,410.775726,429.561157
4,316.940430,321.322479,312.459961,289.940369,339.978027,313.082031,269.146759,291.738556,299.233887,287.862701,301.334839,318.442261,332.436920,262.969971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2742,-17.943813,-31.023998,3.485899,-3.063752,-9.759534,-10.391276,0.315815,-20.167011,-12.834303,3.893237,-1.817984,-9.942732,-3.232556,-11.070627
2743,21.623636,19.183163,30.437355,22.707184,50.872375,32.648174,36.711399,5.690982,12.199239,36.850460,21.653168,43.062523,28.481863,10.198749
2744,9.270197,15.051945,43.288490,17.560698,28.757645,34.593761,40.148769,22.016291,20.840620,38.529934,27.787848,39.001499,42.570694,4.550988
2745,-30.442875,-14.454101,-5.569744,4.859628,14.391598,-12.174311,-21.337261,-20.791552,-11.594651,3.642024,8.604402,1.147189,-20.526121,-28.867086


In [263]:
df_solucion.clip(lower=0, inplace=True)
df_solucion

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,337.455017,278.824097,275.389282,332.793732,334.838531,296.068909,285.253143,281.926971,276.862732,268.698914,334.197815,339.409515,317.949280,281.168976
1,7.742674,4.521966,6.742539,4.463351,0.957877,5.990738,7.986382,3.660743,1.675190,3.162817,3.271239,2.166469,5.363275,6.587953
2,33.249012,34.916206,35.375816,30.073835,29.788906,26.057459,33.799553,26.341917,28.113489,30.577930,28.227587,25.478243,24.677412,27.468515
3,445.453156,344.273193,336.236267,355.775665,403.568481,366.044708,405.181213,447.589172,375.764191,356.018524,357.474701,429.301514,410.775726,429.561157
4,316.940430,321.322479,312.459961,289.940369,339.978027,313.082031,269.146759,291.738556,299.233887,287.862701,301.334839,318.442261,332.436920,262.969971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2742,0.000000,0.000000,3.485899,0.000000,0.000000,0.000000,0.315815,0.000000,0.000000,3.893237,0.000000,0.000000,0.000000,0.000000
2743,21.623636,19.183163,30.437355,22.707184,50.872375,32.648174,36.711399,5.690982,12.199239,36.850460,21.653168,43.062523,28.481863,10.198749
2744,9.270197,15.051945,43.288490,17.560698,28.757645,34.593761,40.148769,22.016291,20.840620,38.529934,27.787848,39.001499,42.570694,4.550988
2745,0.000000,0.000000,0.000000,4.859628,14.391598,0.000000,0.000000,0.000000,0.000000,3.642024,8.604402,1.147189,0.000000,0.000000


In [264]:
df_1_7_daily = df_solucion.iloc[:,0:7]
df_1_7_daily

,0,1,2,3,4,5,6
0,337.455017,278.824097,275.389282,332.793732,334.838531,296.068909,285.253143
1,7.742674,4.521966,6.742539,4.463351,0.957877,5.990738,7.986382
2,33.249012,34.916206,35.375816,30.073835,29.788906,26.057459,33.799553
3,445.453156,344.273193,336.236267,355.775665,403.568481,366.044708,405.181213
4,316.940430,321.322479,312.459961,289.940369,339.978027,313.082031,269.146759
...,...,...,...,...,...,...,...
2742,0.000000,0.000000,3.485899,0.000000,0.000000,0.000000,0.315815
2743,21.623636,19.183163,30.437355,22.707184,50.872375,32.648174,36.711399
2744,9.270197,15.051945,43.288490,17.560698,28.757645,34.593761,40.148769
2745,0.000000,0.000000,0.000000,4.859628,14.391598,0.000000,0.000000


In [265]:
df_1_7_week = df_solucion.iloc[:,0:7].sum(axis=1)
df_1_7_week

0       2140.622711
1         38.405527
2        223.260788
3       2656.532684
4       2162.870056
           ...     
2742       3.801714
2743     214.183287
2744     188.671505
2745      19.251225
2746      37.177915
Length: 2747, dtype: float64

In [266]:
df_8_14_week = df_solucion.iloc[:,7:14].sum(axis=1)
df_8_14_week

0       2100.214203
1         25.887686
2        190.885094
3       2806.484985
4       2094.019135
           ...     
2742       3.893237
2743     158.136983
2744     195.297873
2745      13.393615
2746      25.349684
Length: 2747, dtype: float64

In [267]:
df_final = pd.DataFrame(df_input['ID'])
df_final['2020-02-01'] = df_1_7_daily[0]
df_final['2020-02-02'] = df_1_7_daily[1]
df_final['2020-02-03'] = df_1_7_daily[2]
df_final['2020-02-04'] = df_1_7_daily[3]
df_final['2020-02-05'] = df_1_7_daily[4]
df_final['2020-02-06'] = df_1_7_daily[5]
df_final['2020-02-07'] = df_1_7_daily[6]
df_final['Week 1'] = df_1_7_week
df_final['Week 2'] = df_8_14_week
df_final

,ID,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,Week 1,Week 2
0,0,337.455017,278.824097,275.389282,332.793732,334.838531,296.068909,285.253143,2140.622711,2100.214203
1,1,7.742674,4.521966,6.742539,4.463351,0.957877,5.990738,7.986382,38.405527,25.887686
2,2,33.249012,34.916206,35.375816,30.073835,29.788906,26.057459,33.799553,223.260788,190.885094
3,3,445.453156,344.273193,336.236267,355.775665,403.568481,366.044708,405.181213,2656.532684,2806.484985
4,4,316.940430,321.322479,312.459961,289.940369,339.978027,313.082031,269.146759,2162.870056,2094.019135
...,...,...,...,...,...,...,...,...,...,...
2742,2746,0.000000,0.000000,3.485899,0.000000,0.000000,0.000000,0.315815,3.801714,3.893237
2743,2747,21.623636,19.183163,30.437355,22.707184,50.872375,32.648174,36.711399,214.183287,158.136983
2744,2748,9.270197,15.051945,43.288490,17.560698,28.757645,34.593761,40.148769,188.671505,195.297873
2745,2749,0.000000,0.000000,0.000000,4.859628,14.391598,0.000000,0.000000,19.251225,13.393615


In [268]:
df_final.to_csv("solucion1.csv", sep='|', index=False, header=False )

In [269]:
df_final.to_csv("solucion1.txt", sep='|', index=False, header=False )